In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
import shutil
import zipfile
import kaggle
import tensorflow as tf
from keras.applications.mobilenet_v2 import preprocess_input
from keras.applications import MobileNetV2
from keras import utils
from keras import layers
from keras.models import Model, load_model, Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [ ]:
os.makedirs("/root/.config/kaggle", exist_ok=True)
shutil.move("kaggle.json", "/root/.config/kaggle/kaggle.json")
os.chmod("/root/.config/kaggle/kaggle.json", 600)

kaggle.api.authenticate()
!kaggle datasets download -d grassknoted/asl-alphabet

zip_path = 'asl-alphabet.zip'
extract_to = 'asl-alphabet'
os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print("Pliki zostały rozpakowane pomyślnie!")

In [ ]:
train_dataset, val_dataset = utils.image_dataset_from_directory(
    "./asl-alphabet/asl_alphabet_train/asl_alphabet_train",
    image_size=(224, 224),
    interpolation="nearest",
    validation_split=0.2,
    label_mode="categorical",
    subset="both",
    batch_size=32,
    seed=42,
)

class_names = train_dataset.class_names
print(class_names)

In [ ]:
test_dir = "./asl-alphabet/asl_alphabet_test/asl_alphabet_test"

image_paths = []
labels = []
for fname in os.listdir(test_dir):
    label = fname.split("_")[0]
    image_paths.append(os.path.join(test_dir, fname))
    labels.append(label)

le = OneHotEncoder(sparse_output=False)
le.fit(np.array(class_names).reshape(-1, 1))
y_test = le.transform(np.array(labels).reshape(-1, 1))


def load_image(path):
    img = utils.load_img(path, target_size=(224, 224))
    arr = utils.img_to_array(img)
    return preprocess_input(arr)


x_test = np.array([load_image(path) for path in image_paths])

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
data_augmentation = Sequential([
    layers.RandomTranslation(0.1, 0.1, fill_mode="nearest"),
    layers.RandomZoom(height_factor=0.2, fill_mode="nearest"),
    layers.RandomFlip("horizontal"),
    layers.RandomBrightness(factor=(-0.1, 0.5)),
])


def preprocess_and_augment(image, label):
    image = data_augmentation(image)
    image = preprocess_input(image)
    return image, label


train_dataset = train_dataset.map(preprocess_and_augment).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.map(lambda x, y: (preprocess_input(tf.cast(x, tf.float32)), y)).prefetch(tf.data.AUTOTUNE)

In [ ]:
pretrained_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    pooling="avg"
)
pretrained_model.trainable = False

In [ ]:
inputs = layers.Input(shape=(224, 224, 3))
outputs = pretrained_model(inputs, training=False)
outputs = layers.Dense(128, activation="relu")(outputs)
outputs = layers.Dropout(0.2)(outputs)
outputs = layers.Dense(128, activation="relu")(outputs)
outputs = layers.Dense(29, activation="softmax")(outputs)

model = Model(inputs, outputs)
model.compile(optimizer=Adam(), loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

In [ ]:
early_stopping = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
history = model.fit(train_dataset, validation_data=val_dataset, epochs=10,
                    callbacks=[early_stopping])
model.save("model-train-augment.keras")

In [ ]:
train_acc = history.history["accuracy"]
train_loss = history.history["loss"]
val_acc = history.history["val_accuracy"]
val_loss = history.history["val_loss"]

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(train_acc, label="train_acc")
ax[0].plot(val_acc, label="val_acc")
ax[0].legend()
ax[1].plot(train_loss, label="train_loss")
ax[1].plot(val_loss, label="val_loss")
ax[1].legend()
plt.show()

In [ ]:
model = load_model("model-train-augment-finetuned.keras")

In [ ]:
test_preds = model.evaluate(test_dataset, verbose=0)
print(f'Test accuracy: {test_preds[1] * 100}')
print(f'Test loss: {test_preds[0]}')

In [ ]:
test_preds = model.evaluate(val_dataset, verbose=0)
print(f'Test accuracy: {test_preds[1] * 100}')
print(f'Test loss: {test_preds[0]}')

In [ ]:
FINE_TUNE_AT = 100

for layer in model.layers[1].layers[FINE_TUNE_AT:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-6),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

#model.summary(show_trainable=True)

early_stopping = EarlyStopping(monitor="val_loss", patience=2, restore_best_weights=True)
finetuned_history = model.fit(train_dataset, validation_data=val_dataset, epochs=20, initial_epoch=len(history.epoch),
                              callbacks=[early_stopping])

In [ ]:
model.save("model-train-augment-finetuned.keras")

In [ ]:
combined_history = history.history.copy()
for key in combined_history.keys():
    combined_history[key] = combined_history[key] + finetuned_history.history[key]

train_acc = combined_history["accuracy"]
train_loss = combined_history["loss"]
val_acc = combined_history["val_accuracy"]
val_loss = combined_history["val_loss"]

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].plot(train_acc, label="train_acc")
ax[0].plot(val_acc, label="val_acc")
ax[0].legend()
ax[1].plot(train_loss, label="train_loss")
ax[1].plot(val_loss, label="val_loss")
ax[1].legend()
plt.show()

In [ ]:
def reverse_mobilenet_transform(img):
    return (img + 1) * 127.5


fig, axes = plt.subplots(4, 7, figsize=(20, 12))
index_to_class = {i: cls for i, cls in enumerate(class_names)}
print(index_to_class)
for images, labels in test_dataset.take(1):
    for i, ax in enumerate(axes.flat):
        img = reverse_mobilenet_transform(images[i]) / 255.0
        ax.imshow(img)
        ax.axis('off')

        img_to_predict = np.expand_dims(images[i], axis=0)
        pred = np.argmax(model.predict(img_to_predict))
        predicted_label = index_to_class[pred]
        ax.set_title(f'True label: {index_to_class[np.argmax(labels[i])]}\nPred label: {predicted_label}')

plt.show()